In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing Libraries

In [2]:
import glob
import cv2
import re
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
#import tqdm as tqdm
#from tqdm.notebook import tqdm

from tensorflow import keras

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.models import load_model
from tensorflow.keras import utils
#from keras.utils.np_utils import to_categorical 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import DirectoryIterator
from tensorflow.keras.applications.resnet import preprocess_input

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# from tensorflow.keras.applications.resnet import ResNet101
# from tensorflow.keras.applications.resnet_v2 import ResNet101V2
# from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

from sklearn.model_selection import train_test_split

### Read data

In [3]:
data = pd.read_csv("/content/drive/My Drive/Data/spotle_hack/aithon2020-level-2/data/aithon2020_level2_traning.csv")

In [4]:
data.head()

,emotion,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,pixel_10,pixel_11,pixel_12,pixel_13,pixel_14,pixel_15,pixel_16,pixel_17,pixel_18,pixel_19,pixel_20,pixel_21,pixel_22,pixel_23,pixel_24,pixel_25,pixel_26,pixel_27,pixel_28,pixel_29,pixel_30,pixel_31,pixel_32,pixel_33,pixel_34,pixel_35,pixel_36,pixel_37,pixel_38,...,pixel_2264,pixel_2265,pixel_2266,pixel_2267,pixel_2268,pixel_2269,pixel_2270,pixel_2271,pixel_2272,pixel_2273,pixel_2274,pixel_2275,pixel_2276,pixel_2277,pixel_2278,pixel_2279,pixel_2280,pixel_2281,pixel_2282,pixel_2283,pixel_2284,pixel_2285,pixel_2286,pixel_2287,pixel_2288,pixel_2289,pixel_2290,pixel_2291,pixel_2292,pixel_2293,pixel_2294,pixel_2295,pixel_2296,pixel_2297,pixel_2298,pixel_2299,pixel_2300,pixel_2301,pixel_2302,pixel_2303
0,Fear,231,212,156,164,174,138,161,173,182,200,106,38,39,74,138,161,164,179,190,201,210,216,220,224,222,218,216,213,217,220,220,218,217,212,174,160,162,160,139,...,225,220,215,207,199,167,108,151,122,88,71,84,120,127,105,76,71,78,90,106,123,146,155,148,130,141,119,69,54,89,104,138,152,122,114,101,97,88,110,152
1,Fear,55,55,55,55,55,54,60,68,54,85,151,163,170,179,181,185,188,188,191,196,189,194,198,197,195,194,190,193,195,184,175,172,161,159,158,159,147,136,137,...,64,90,120,121,127,136,128,150,154,157,155,167,172,174,173,171,167,163,164,160,155,148,148,144,130,126,124,116,118,110,90,83,77,53,37,34,30,34,30,57
2,Sad,20,17,19,21,25,38,42,42,46,54,56,62,63,66,82,108,118,130,139,134,132,126,113,97,126,148,157,161,155,154,154,164,189,204,194,168,180,188,214,...,63,42,43,76,80,76,73,69,64,59,53,54,52,54,57,60,66,71,77,85,89,93,102,106,105,99,85,62,34,22,29,36,68,91,85,93,97,99,107,118
3,Happy,4,2,13,41,56,62,67,87,95,62,65,70,80,107,127,149,153,150,165,168,177,187,176,167,152,128,130,149,149,146,130,139,139,143,134,105,78,56,36,...,19,16,11,8,46,47,46,47,48,51,55,55,56,76,85,84,84,77,86,86,80,79,82,84,76,62,57,62,61,62,59,66,80,74,69,99,28,3,7,12
4,Fear,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,...,43,57,66,74,76,76,82,87,97,72,30,60,68,78,72,63,46,80,80,35,39,48,65,90,89,53,17,37,44,82,94,21,22,51,71,80,82,79,79,83


In [5]:
data.shape

(10817, 2305)

In [6]:
#X, y_cat = data_xy("rgb")

### Read the data as grayscale images

In [7]:
X = []
y_cat= []
for tup in data.itertuples():
      image = np.array(tup[2:]).reshape(48,48).astype(float).tolist()
      X.append(image)
      y_cat.append(tup[1])
X = np.array(X)#.reshape(-1,48,48,1)

In [8]:
### One hot encoding of Label
# print("Unique labels:",set(y_cat))
# convert_array = {'Happy': [1,0,0],'Sad': [0,1,0],'Fear':[0,0,1] }
# # X = np.array(X).reshape(-1,48,48,1)
# #y_encoded = np.array([convert_array[i] for i in y_cat])
# #y = np.eye(len(set(y_encoded)))[y_encoded]
# y = np.array([convert_array[i] for i in y_cat])
# y.shape

### Check no of data points corresponding to each class

In [9]:
dic = {}
for key in y_cat:
  try:
    dic[key] += 1
  except:  
    dic[key] = 1

In [10]:
for key in dic.keys():
  print("No of images for category {} : {} ".format(key, dic[key]))

No of images for category Fear : 2737 
No of images for category Sad : 3271 
No of images for category Happy : 4809 


### Convert Gray scale to RGB (by repeating the same channel)

In [ ]:
X_rgb = []
for item in X:
      X_rgb.append(np.stack((item,)*3,axis = -1).tolist())
X_rgb = np.array(X_rgb)

### Resize the RGB image (as required for the pre-trained model)

In [12]:
width = 80
height = 80
dim = (width, height)
 
temp  = X_rgb
X = []
for i in range(len(temp)):
  X.append(cv2.resize(temp[i],dim))
X = np.array(X) 

### Train-Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y_cat, test_size=0.30, random_state=42, shuffle=True)

### Oversampling for training data



In [14]:
train_dic = {}
for i in range(len(y_train)):
  try:
    train_dic[y_train[i]].append(X_train[i])
  except:  
    train_dic[y_train[i]] = [X_train[i]]

In [15]:
for key in train_dic.keys():
  print("No of images for category {} in training data: {} ".format(key, len(train_dic[key])))

No of images for category Happy in training data: 3361 
No of images for category Fear in training data: 1934 
No of images for category Sad in training data: 2276 


In [16]:
m = max([len(train_dic[key]) for key in train_dic.keys()])
for key in train_dic.keys():
     l =(m - len(train_dic[key]))
     if l!= 0:
      #print(np.array(random.choices(dic[key],k=l)).shape)
      X_train = np.concatenate((X_train, np.array(random.choices(train_dic[key],k=l))))
      y_train = np.concatenate((y_train,np.array([key]*l)),axis=0)
      #y = np.concatenate((y,np.stack((np.array(convert_array[key]),)*l)),axis=0)

In [17]:
print(X_train.shape)
print(y_train.shape)

(10083, 80, 80, 3)
(10083,)


### One hot encoding of label

In [18]:
print("Unique labels:",set(y_cat))
convert_array = {'Happy': [1,0,0],'Sad': [0,1,0],'Fear':[0,0,1] }
# X = np.array(X).reshape(-1,48,48,1)
#y_encoded = np.array([convert_array[i] for i in y_cat])
#y = np.eye(len(set(y_encoded)))[y_encoded]
y_train = np.array([convert_array[i] for i in y_train])
y_test = np.array([convert_array[i] for i in y_test])

Unique labels: {'Sad', 'Fear', 'Happy'}


In [19]:
print("No of training data:",X_train.shape[0])
print("No of validation data:",X_test.shape[0])

No of training data: 10083
No of validation data: 3246


### Define Model

In [20]:
IMAGE_SIZE = 80

base_model = VGG16(include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), weights="imagenet") 
 
base_model.trainable = True


model = Sequential([base_model])
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256, activation = "relu"))
#model.add(BatchNormalization())
model.add(Dropout(0.25))
# model.add(Dense(128, activation = "relu"))
# model.add(Dropout(0.5))
model.add(Dense(3, activation = "softmax"))

optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
#optimizer = RMSprop(learning_rate=0.001)


model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [21]:
def fit_model(model,epochs = 50, batch_size = 64, data_augmentation = False):

    rlr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience= 3, verbose=1, mode='auto', min_lr=0)

    if not data_augmentation:
      print("Fitting model without using Data Augmenatation.")
      history = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True, verbose = 2, callbacks=[rlr])
    else:
      print("Fitting model using Data Augmenatation.")
      datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=True,  # set each sample mean to 0
        featurewise_std_normalization=True,  # divide inputs by std of the dataset
        samplewise_std_normalization=True,  # divide each input by its std
        zca_whitening=True,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.,
        shear_range=0.,  # set range for random shear
        zoom_range=0.1,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied befor/e any other transformation)
        rescale=None)
      
      datagen.fit(X_train)
      history = model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        shuffle = True,verbose = 2, callbacks=[rlr])
    return history

### Fit the model

In [1]:
history = fit_model(model, data_augmentation=False)

### For prediction 

In [ ]:
model.predict()